In [3]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

# --- CONFIG ---
URL = "https://filingaccess.serff.com/sfa/home/CO"
TABLE_BODY = "tbody#j_idt25\\:filingTable_data"
WAIT_TIME = 15
SCRAPE_INTERVAL = 5  # seconds between scrapes
CSV_FILE = "insurance_filings_manual_pagination_v2.csv"

options = webdriver.ChromeOptions()
options.add_argument("--start-maximized")

driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
driver.get(URL)

# --- Wait for manual filter selection ---
print("⏳ Please set filters manually within 20 seconds...")
time.sleep(20)

wait = WebDriverWait(driver, WAIT_TIME)

# --- Table headers ---

# --- Helper to extract current table page and page number ---
def extract_table_with_page():
    wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, f"{TABLE_BODY} tr")))

    # Get current page number
    try:
        current_page_text = driver.find_element(By.CSS_SELECTOR, "span.ui-paginator-current").text
        # Example: "(1 of 1684)" → extract "1"
        current_page = int(current_page_text.split(" ")[0].replace("(", ""))
    except:
        current_page = None

    rows = driver.find_elements(By.CSS_SELECTOR, f"{TABLE_BODY} tr")
    data = []
    for r in rows:
        cols = r.find_elements(By.TAG_NAME, "td")
        row_data = [c.text.strip() for c in cols]
        row_data.append(current_page)  # append current page number
        data.append(row_data)

    return data

headers = [
    "Company Name",
    "NAIC Company Code",
    "Insurance Product Name",
    "Sub Type Of Insurance",
    "Filing Type",
    "Filing Status",
    "SERFF Tracking Number",
    "Page Number"   # new column
]


all_data = []

print("🚀 Scraping loop started. Manually click 'Next' when ready.")

try:
    while True:
        # --- Extract table safely ---
        try:

            page_data = extract_table_with_page()

            if page_data:
                all_data.extend(page_data)
                # --- Save CSV intermittently ---
                df = pd.DataFrame(all_data, columns=headers)
                df.to_csv(CSV_FILE, index=False)
                print(f"Scraped {len(page_data)} rows. Total rows: {len(all_data)}. CSV saved.")
            else:
                print("No data found on this page.")
        except Exception as e:
            print("⚠️ Error extracting table:", e)

        time.sleep(SCRAPE_INTERVAL)

except KeyboardInterrupt:
    # --- Save CSV on exit ---
    df = pd.DataFrame(all_data, columns=headers)
    df.to_csv(CSV_FILE, index=False)
    print(f"\n✅ Done! Total rows scraped: {len(all_data)}.")
    driver.quit()


⏳ Please set filters manually within 20 seconds...
🚀 Scraping loop started. Manually click 'Next' when ready.
Scraped 20 rows. Total rows: 20. CSV saved.
Scraped 100 rows. Total rows: 120. CSV saved.
Scraped 100 rows. Total rows: 220. CSV saved.
Scraped 100 rows. Total rows: 320. CSV saved.
Scraped 100 rows. Total rows: 420. CSV saved.
Scraped 100 rows. Total rows: 520. CSV saved.
Scraped 100 rows. Total rows: 620. CSV saved.
Scraped 100 rows. Total rows: 720. CSV saved.
Scraped 100 rows. Total rows: 820. CSV saved.
Scraped 100 rows. Total rows: 920. CSV saved.
Scraped 100 rows. Total rows: 1020. CSV saved.
Scraped 100 rows. Total rows: 1120. CSV saved.
Scraped 100 rows. Total rows: 1220. CSV saved.
Scraped 100 rows. Total rows: 1320. CSV saved.
Scraped 100 rows. Total rows: 1420. CSV saved.
Scraped 100 rows. Total rows: 1520. CSV saved.
Scraped 100 rows. Total rows: 1620. CSV saved.
Scraped 100 rows. Total rows: 1720. CSV saved.
Scraped 100 rows. Total rows: 1820. CSV saved.
Scraped 1